# Scoring function for SBA loan approval model
We will create a scoring function for the SBA loan approval model

In [8]:
def project_1_scoring(data):
    
    '''Returns model performance scores given a dataset'''
    import pandas as pd
    pd.set_option('display.max_columns', 1500)

    import warnings
    warnings.filterwarnings('ignore')

    #Extend cell width
    from IPython.core.display import display, HTML
    display(HTML("<style>.container { width:80% !important; }</style>"))
    
    import numpy as np
    import category_encoders as ce
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.preprocessing import StandardScaler
    from sklearn.linear_model import LogisticRegression
    import pickle

    
    # Convert dataset to pandas dataframe
    data = pd.DataFrame(data)
    
    
    # Drop MIS_Status column if it exists
    if 'MIS_Status' in data.columns: 
        data.drop('MIS_Status', axis=1, inplace=True)
        
    # Load Artifacts:
    with open("TTK210000_artifacts_dict.pkl", "rb") as f:
        TTK210000_artifacts_dict = pickle.load(f)
        
    # Access the contents of the dictionary
        best_lreg = TTK210000_artifacts_dict["model"]
        ohe = TTK210000_artifacts_dict["one_hot_encoder"]
        woe_encoder = TTK210000_artifacts_dict["woe_encoder"]
        scaler = TTK210000_artifacts_dict["Min_Max_Scaler"]
        best_threshold = TTK210000_artifacts_dict["threshold"]

    # Convert the strings styled as '$XXXX.XX' to float values.
    float_nums = ['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']
    for column in float_nums:
        data[column] = pd.to_numeric(data[column].str.replace('$', '').str.replace(',', ''))
        
    #Numeric and Categorical Columns
    num_cols = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
    cat_cols = data.select_dtypes(include=['object']).columns.tolist()
    
    # Filling Numeric column with 0
    data[num_cols]=data[num_cols].fillna(0)
    
    # Filling categorical columns with "Missing"
    data[cat_cols] = data[cat_cols].fillna("Missing")
    
    # Feature Engineering to add 10 new features
    
    # Feature 1: OutstandingBalance - ratio of BalanceGross and DisbursementGross
    data['OutstandingBalance'] = data['BalanceGross'] / data['DisbursementGross']
    
    # Feature 2: IsFranchise - Convert 0 for not franchise and 1 for franchise
    data['IsFranchise'] = data['FranchiseCode'].apply(lambda x: 0 if x in [0, 1] else 1)
    data = data.drop('FranchiseCode', axis=1)
    
    # Feature 3: SBAAppvRate - how much of Gross approved amount is SBA approved
    data['SBAAppvRate'] = data['SBA_Appv'] / data['GrAppv']
    
    # Feature 4: DisbursementRate - how much of approved amount was disbursed
    data['DisbursementRate'] = data['DisbursementGross'] / data['GrAppv']
    
    # Feature 5: JobLoss - Job Loss will be 1, and no job loss will be 0
    data['JobLoss'] = data['NoEmp'] + data['CreateJob'] - data['RetainedJob']
    data['JobLoss'] = data['JobLoss'].apply(lambda x: 1 if x < 0 else 0 )
    
    # Feature 6: Categorize short, medium, long term loans 
    data['ShortTerm'] = data['Term'].apply(lambda x: 1 if x < 18 else 0)
    data['MediumTerm'] = data['Term'].apply(lambda x: 1 if 18 <= x <= 60 else 0)
    data['LongTerm'] = data['Term'].apply(lambda x: 1 if x > 60 else 0)
    data = data.drop(['Term'], axis=1)
    
    # Feature 7: Categorize based on loan size - small, medium and large loan
    data['SmallLoan'] = data['DisbursementGross'].apply(lambda x: 1 if x < 100000 else 0)
    data['MediumLoan'] = data['DisbursementGross'].apply(lambda x: 1 if 100000 <= x <= 450000 else 0)
    data['LargeLoan'] = data['DisbursementGross'].apply(lambda x: 1 if x > 450000 else 0)
    
    # Feature 8: SameState - checks if Bank is in the same state as the lender bank
    data['SameState'] = data.apply(lambda x: 0 if x['State'] == x['BankState'] else 1, axis = 1)
    
    # Feature 9: Industry -take first 2 digits of NAICS code
    data['Industry'] = data['NAICS'].astype('str').apply(lambda x: x[:2])
    data = data.drop('NAICS', axis=1)
    
    # Feature 10: Regions - Map states into Northeast, Midwest, South, West
    
    # Create region mapping function
    def map_region(state):
        if state in ['CT', 'ME', 'MA', 'NH', 'RI', 'VT', 'NJ', 'NY', 'PA']:
            return 'Northeast'
        elif state in ['IL', 'IN', 'IA', 'KS', 'MI', 'MN', 'MO', 'NE', 'ND', 'OH', 'SD', 'WI']:
            return 'Midwest'
        elif state in ['AL', 'AR', 'DE', 'FL', 'GA', 'KY', 'LA', 'MD', 'MS', 'NC', 'OK', 'SC', 'TN', 'TX', 'VA', 'WV']:
            return 'South'
        elif state in ['AK', 'AZ', 'CA', 'CO', 'HI', 'ID', 'MT', 'NV', 'NM', 'OR', 'UT', 'WA', 'WY']:
            return 'West'
        else:
            return 'Unknown'
    # Map regions into dataset 
    data['Region'] = data['State'].apply(map_region)
    
        
    # WOE encoding
    woe_cols = ['City','State','Bank','BankState','RevLineCr','Industry']
    
    # transform data to WOE encoding
    
    data_woe = woe_encoder.transform(data)
    
    # Rename the WOE encoded columns:
    
    data_woe.rename(columns={'City':'City_woe', 'State':'State_woe', 'Bank':'Bank_woe',
                           'BankState':'BankState_woe', 'RevLineCr':'RevLineCr_woe',
                           'Industry':'Industry_woe'}, inplace=True)
    
    
    # Add the WOE Encoded column to original dataset
    data[['City_woe', 'State_woe', 'Bank_woe', 'BankState_woe',
        'RevLineCr_woe','Industry_woe']]=data_woe[['City_woe', 'State_woe', 'Bank_woe', 'BankState_woe',
                                               'RevLineCr_woe','Industry_woe']]
    
    
    # Drop the original columns
    data = data.drop(columns = ['City','State','Bank','BankState','RevLineCr'])
           
    # One hot encoding
    cols_to_encode = ['LowDoc', 'Region']
    
    # transform one hot encoder on dataset 
    new_col_names = ohe.transform(data[cols_to_encode]).astype(int).astype(str)
    new_col_names = [col + '_' + name for col, names in zip(cols_to_encode, ohe.categories_) for name in names]
    
    #Apply OHE to dataframe
    ohe_encoded = pd.DataFrame(ohe.transform(data[cols_to_encode]).astype(int), columns=new_col_names, index=data.index)
    
    # add encoded columns to original dataset
    data = pd.concat([data, ohe_encoded], axis=1)
    
    # drop the original columns
    data = data.drop(columns=cols_to_encode)
    
    # Drop index column
    record_id = data['index']
    data = data.drop('index', axis = 1)
    
    data = data.astype({'Zip':'str', 'NewExist': 'int64'})
    
    # Fill NA values after encoding
    values_to_fill = {}
    for col in data.columns:
        if pd.api.types.is_numeric_dtype(data[col].dtype):
            values_to_fill[col] = 0
        else:
            values_to_fill[col] = "Missing"
        
    data.fillna(value=values_to_fill,inplace=True)

    # Scaling dataset
    scaler_cols = ['NoEmp','CreateJob','RetainedJob','DisbursementGross','BalanceGross','GrAppv','SBA_Appv']
    
    
    # transform
    data_scaled = scaler.transform(data[scaler_cols])
    
    
    # Convert to dataframe
    scaled_data = pd.DataFrame(data_scaled,
                             columns=['NoEmp_scaled','CreateJob_scaled','RetainedJob_scaled',
                                      'DisbursementGross_scaled','BalanceGross_scaled',
                                      'GrAppv_scaled','SBA_Appv_scaled'])
    
    
    # add scaled columns to original dataset
    data= pd.concat([data.reset_index(drop=True),scaled_data], axis =1)
    
    
    # drop the original columns
    data = data.drop(columns=scaler_cols)
       
    # Get the probability threshold to maximize F1 on the dataset
    y_pred = (best_lreg.predict_proba(data)[:,1] >=best_threshold).astype(int)
    y_pred_proba = best_lreg.predict_proba(data)
    
    d = {"index": record_id,
         "label":y_pred,
         "probability_0":y_pred_proba[:,0],
         "probability_1":y_pred_proba[:,1]}
    
    return pd.DataFrame(d)
    

In [2]:
import pandas as pd
sba_df = pd.read_csv('SBA_loans_project_1.zip')

sba_df = sba_df.sample(frac=0.05)

In [9]:
project_1_scoring(sba_df)

,index,label,probability_0,probability_1
198206,198206,1,0.525320,0.474680
364208,364208,0,0.948636,0.051364
17505,17505,1,0.569269,0.430731
258624,258624,0,0.977369,0.022631
29264,29264,0,0.982075,0.017925
...,...,...,...,...
732986,732986,0,0.807468,0.192532
620999,620999,1,0.466503,0.533497
86520,86520,1,0.388222,0.611778
753077,753077,0,0.945775,0.054225
